In [ ]:
# 下準備
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null # mecabの利用に必要なライブラリのインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null                    # gitから辞書ファイルのクローン
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1                   # クローンした辞書のインストール
!pip install mecab-python3==0.7 > /dev/null                                                             # 0.7以外だと謎のエラーが発生して安定しないことがある

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc                                                              # 辞書の参照先にインストール先のディレクトリを追加
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd" 

!pip install fastapi uvicorn nest-asyncio pyngrok                                                       # 追記
!pip install jinja2

In [ ]:
"""
下準備
"""
# ドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

!mkdir /templates
!cp /content/drive/MyDrive/python_ml_intro/app/class_app_index.html /templates/index.html
!cat templates/index.html

In [ ]:
# アプリUI画面の配置
!mkdir templates
!curl -v -o templates/index.html https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/app/class_app_index.html
!ls templates/
!cat templates/index.html

In [ ]:
import numpy as np
import json
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import MeCab
import logging

class ClassAppSelect():

  def __init__(self) -> None:
    logger = logging.getLogger("uvicorn")
    logger.info("initialize start.")

    self.tokenizer = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")
    with open("/content/drive/MyDrive/python_ml_intro/data/class_select_app/content_info_dict.json","r") as f:
      self.content = json.load(f)
    
    self.wv = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/python_ml_intro/data/class_select_app/wiki_word_vec.pt",binary=True)
    self.content_vec = np.load("/content/drive/MyDrive/python_ml_intro/data/class_select_app/content_vec.npy")
    
    logger.info("initialize done.")


  def generate_query(self,query: str) -> np.ndarray:
    sum = np.zeros(300)                                    # 授業の文章ベクトルを格納する要素を0とした要素数300ノベクトルを初期化。
    words = self.tokenizer.parse(query)[:-1].split(" ")    # 集約した文章を単語のリストに分割。mecabで分割した際には単語間にスペースが入った文字列として出力。最後の改行は邪魔なので考慮していない。
    recg_word_num = 0                                      # 文章内で認識できた単語の数を数える

    for word in words:
      try:
        sum += self.wv[word]
        recg_word_num += 1
      except KeyError:                                     # 学習していない単語の場合は考慮しない
        pass
      
    # もし学習済みの単語がない場合はランダムなベクトルを割り当てる
    if recg_word_num == 0:
      return np.random(300)
    else:
      return sum / recg_word_num


  def culculate_sim(self,query :np.ndarray) -> int:
    input_query = self.generate_query(query)
    sim_rate = cosine_similarity([input_query],self.content_vec)
    return self.content[np.argmax(sim_rate)]

In [ ]:
test = ClassAppSelect()

In [ ]:
print(test.culculate_sim("キャリアプラン"))

{'教科名': '情報と職業', '担当者': '松村\u3000哲哉', '授業目的': '「社会人として仕事をする」ということの目的、意義を理解し、キャリアプランを立案する方法を習得する。また、情報系企業の業種分野ごとに、企業の取り組みと技術者に求められる技術力と社会人基礎力について理解し、今後の学生生活の中で身につけておくべき技術力と社会人基礎力を向上させる対策を立案する,', '教育目標': '（１）「社会人として仕事をする」ということの目的、意義を理解し、キャリアプランを立案する方法を習得できる。。（２）将来、自分にとって本当に望ましい企業に就職するための準備として、情報系企業の業種分野ごとに、将来の動向、解決すべき課題、課題解決に向けての取り組みと技術者に求められる技術力と社会人基礎力について理解できる．（３）自分の就職する企業を想定して、志望理由と自分が所有すべき技術力と社会人基礎力を検討し、今後の学生生活の中で身につけておくべき技術力と社会人基礎力を向上させる対策を立案できる．', '概要': '授業は、キャリアデザインの講義を行い、自己分析等を通じて自分の将来設計についての手法を学ぶ。また、６名の社会人講師（情報関連企業に勤務）から、企業の業種分野ごとに将来の動向、解決すべき課題、課題解決に向けての取り組みと技術者に求められる技術力と社会人基礎力のお話を伺い、その後、就職する企業を想定して、志望理由と自分が所有すべき技術力と社会人基礎力を学ぶ。', '成績評価': '評価は絶対評価とし、授業内試験（１）で50％、講義毎のレポートで50％の成績評価を行い、総合して100点満点で採点する．合計60点以上を合格とする．', '課題': 'レポートの採点結果の点数については随時開示する．'}


In [ ]:
from fastapi import FastAPI,Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.templating import Jinja2Templates

app = FastAPI()
class_select_app = ClassAppSelect()
templates = Jinja2Templates(directory='templates')

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root(request: Request):
    return templates.TemplateResponse("index.html",{"request":request})

@app.get("/app")
async def user_input(user_req: str) -> dict:
  return class_select_app.culculate_sim(user_req)

In [ ]:
import nest_asyncio
from pyngrok import ngrok,conf
import uvicorn

# 認証トークンの設定。htmlファイルを配信するために必要です。
conf.get_default().auth_token = "1d2GMN6jsGz90I1W8dNizJJUnWy_CqQHkfxVG8zypGbHnsQL"

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)